# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

**Цель исследования** — проверить четыре гипотезы:
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Исследование пройдёт в четыре этапа:
 1. [Изучение данных, предоставленных банком](#intro).
 2. [Предобработка данных](#obrabotka).
 3. [Ответы на поставленные вопросы](#otvety).
 4. [Общий вывод](#vyvod).


## Шаг: Откройте файл с данными и изучите общую информацию <a id='intro'></a>

In [1]:
import pandas as pd
dataset=pd.read_csv('data.csv')
dataset.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Просмотрим общую информацию о таблице:

In [2]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Согласно документации к данным:
    
* children — количество детей в семье;
* days_employed — общий трудовой стаж в днях;
* dob_years — возраст клиента в годах;
* education — уровень образования клиента;
* education_id — идентификатор уровня образования;
* family_status — семейное положение;
* family_status_id — идентификатор семейного положения;
* gender — пол клиента;
* income_type — тип занятости;
* debt — имел ли задолженность по возврату кредитов;
* total_income — ежемесячный доход;
* purpose — цель получения кредита;

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Вывод**


1. Названия столбцов записаны правильно ( в "зменином" регистре, прописными буквами, латиницей, без пробелов).
2. Общий трудовой страж (days_employed) местами указан в отрицательном значение, что не может быть. Была бы возможность надо было бы уточнить у разрабочиков - спросил бы у них, почему так происходит. Предполагаю, что данные отрицательные из-за ошибки выгрузки. Так же не совсем уверен, что все данные представлены в днях ( некоторые числа больше похоже на часы, далее уточним этот момент).
3. Разные регистры написания данных в Датасете. Приведем все записи в нижний регистр далее.
4. Ежемесячный доход (total_income) так же округлю до целых чисел, считаю что на исследование это не повлияет.
5. В Датасете имеются пропуски, так как возможности узнать почему это произошло нет, то далее посмотрим что конкретно за пропуски и изучим их влияние на ход исследования ( если влияние не существенное, то удалим их. если влияение существенное, то решим вопрос на какое значение их заменить)


## Шаг: Предобработка данных <a id='obrabotka'></a>

### Обработка пропусков

Посчитаем, сколько в таблице пропущенных значений:

In [3]:
dataset.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Очень интересно получается, что пропуски в двух столбцах: стаж и ежемесячный доход и их количество равно. Надо будет проверить данные пропуски в одной строке или нет? Если в одной строке - то логично предположить, что данные указаны по потенциальным заемщикам, которые еще не работают и у которых соответственно отсутсвуют доходы ( например, студенты или только закончившие учиться люди). Но возможен и другой вариант: если вдруг была ошибка записи в трудовом стаже, то автоматически появлялся пропуск и в ежемесячном доходе ( опять таки надо уточнить у разработчиков почему так происходит) или наоборот.
Пропущенных значений чуть более 10% от всех, это достаточно много, поэтому удалять их не будем.

Вначале заменим все пропущенные значения в столбцах 'days_employed' и 'total_income' на нулевые значения. И проверим совпадает ли количесво нулевых значений в этих столбцах с первоночальными пропусками. И проверим ушли ли пропуски.

In [4]:
columns_to_replace =[dataset.columns] 
for index in columns_to_replace:
    dataset[index] = dataset[index].fillna('0')
#print(dataset['days_employed'].value_counts())
#print(dataset['total_income'].value_counts())
dataset.isna().sum() 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Сравним пропуски в двух столбцах: в одинковых строкак пропуски или нет:

In [5]:
days_employed_new=dataset['days_employed']
total_income_new=dataset['total_income']
proverka=0 # в переменную будем добавлять по единице, если пропуск в столбце 'days_employed' совпадает с пропуском в столбце 'total_income' 
for index in range(len(days_employed_new)):
    if days_employed_new[index] == total_income_new[index]:
        proverka+=1
print(proverka)        

2174


**Вывод**

В результате проверки выяснилось, что все пропуски содержащиеся в двух столбцах находятся в одинаковых строках по столбцам.

Проверим далее гипотезу о том, что пропуски содеражтся у людей, которые только закончили учиться или еще учатся. ( сделаем это в п. 2.5 Категоризация данных)

Пропуски пока оставим в значениях "0", далее скорее всего по столбцу 'total_income' будем производить замену этих значение в зависимости от Категориизации данных. ( столбец 'days_employed' скорее всего вообще удалим, пока он не несет в себе никакой ценности)

### Замена типа данных

Преобразуем данные в Датафрейма  в колонке 'days_employed' в целые числы и сделаем их положительными:

In [6]:
dataset['days_employed']=abs(dataset['days_employed'].astype('int'))
#dataset.info()
dataset['days_employed'].value_counts()

0         2174
133         16
327         16
438         15
223         14
          ... 
350691       1
329638       1
346609       1
360739       1
6219         1
Name: days_employed, Length: 9087, dtype: int64

Так же преобразуем значения в стоблце 'total_income' в целые числа:

In [7]:
dataset['total_income']=dataset['total_income'].astype('int')
#dataset.info()
dataset['total_income'].value_counts()

0         2174
204827       3
163060       3
144533       3
121931       3
          ... 
109583       1
101387       1
138249       1
280240       1
229304       1
Name: total_income, Length: 18607, dtype: int64

Посмотрим столбец 'children' и узнаем какие данные в нем хранятся:

In [8]:
dataset['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Получили информацию о том, что в столбце 'children' присутствуют отрицательные данные ( что не может быть в принципе, скорее всего техническая ошибка) и слишком большие цифры - 20 детей, что скорее всего является ошибкой при записи ( мое предположение: 2 на дополнительной клавиатуре находится над 0, поэтому когда нажимали цифру 2 случайно задели 0 и не обратили на это внимание. Человеческий фактор, увы от него никуда не деться). Если '- 1'  ребенок в семье это явная ошибка, то с 20 не все так олнозначно. Возможно это действительно семьи с количеством детей 20, но так как 20 детей имеют 76 человек, а это менее 1 % исследуемых человек, то мы можем допустить такую погрешность и исправить на 2 детей в семье. 

Исправим эту ситуацию:

In [9]:
def replace_wrong_children (wrong_children,correct_children): 
    for wrong_childre in wrong_children: 
        dataset['children'] = dataset['children'].replace(wrong_children, correct_children)

false_children_1 = [-1]
true_children_1 = 1
replace_wrong_children(false_children_1, true_children_1)
false_children_20 = [20] 
true_children_20 = 2
replace_wrong_children(false_children_20, true_children_20)
dataset['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод**


Мы заменили дробные числа из столбцов 'total_income' и 'days_employed' на целые числа. Считаю что это никак не повлияет на результаты исследования, так как дробная часть ( которую мы отбросили в результате приведения к целым числам) имела несущественный вес в общем объеме данных.
Так же мы взяли по модулю значения из колонки 'days_employed', так как подозреваю, что при переносе данных в Датафрейм произошла ошибка записи и они были записаны как отрицательные ( все бы это хорошо уточнить у разработчиков, но увы такой возможности нет).
Остальные данные я считаю представленные в нужных типах, которые менять не требуется (пока не требуется).

Так же мы исправили ситуацию с колонкой 'children', в которой хранились отрицательные значения ( что в принципе не может быть) и слишком большие значения ( которых по-моему мнению не должгно быть).

### Обработка дубликатов

Посчитаем явные дубликаты:

In [10]:
print ('Дубликатов в таблице:', dataset.duplicated().sum())

Дубликатов в таблице: 54


Удалим явные дубликаты(с удалением старых индексов и формированием новых):

In [11]:
dataset = dataset.drop_duplicates().reset_index(drop=True)
print ('Дубликатов в таблице:', dataset.duplicated().sum())

Дубликатов в таблице: 0


Изначально просматривая таблицу можно было заметить, что некоторые значения были записаны разными регистрами.
Приведем все значения к нижнему регистру (в тех столбцах где это необходимо):

In [12]:
#print(dataset['education'].value_counts())
dataset['education'] =dataset['education'].str.lower()
#dataset['family_status'].value_counts()
#dataset['income_type'].value_counts()
dataset['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

**Тут я погорячился и сделал без лемматизации**

Я закоментирую то что сделал и попробую то же самое сделать в п. 2.3

Обратим внимание на то, что цель кредита повторяется, просто записана в разных выражениях. Исправим этот момент:

In [13]:
#dataset['purpose'].value_counts()
#def replace_wrong_purposes (wrong_purposes,correct_purposes): 
    #for wrong_purpose in wrong_purposes: 
       # dataset['purpose'] = dataset['purpose'].replace(wrong_purposes, correct_purposes)

false_purpose_wedding = ['на проведение свадьбы', 'сыграть свадьбу']
true_purpose_wedding = 'свадьба'
#replace_wrong_purposes(false_purpose_wedding, true_purpose_wedding)

false_purpose_education = ['заняться высшим образованием', 'дополнительное образование', 'высшее образование',
                           'получение дополнительного образования', 'получение образования', 'профильное образование', 
                           'получение высшего образования', 'заняться образованием']
true_purpose_education = 'образование'
#replace_wrong_purposes(false_purpose_education, true_purpose_education)

false_purpose_auto = ['сделка с подержанным автомобилем', 'на покупку подержанного автомобиля', 'свой автомобиль', 
                      'автомобили', 'приобретение автомобиля', 'сделка с автомобилем', 'на покупку своего автомобиля', 
                      'на покупку автомобиля']
true_purpose_auto = 'автомобиль'
#replace_wrong_purposes(false_purpose_auto, true_purpose_auto)

false_purpose_lodging = ['операции с жильем', 'покупка жилья для сдачи', 'покупка жилья', 'покупка жилья для семьи', 
                         'покупка недвижимости', 'покупка своего жилья', 'ремонт жилью', 'покупка жилой недвижимости',
                         'строительство собственной недвижимости', 'недвижимость', 'операции со своей недвижимостью', 
                         'троительство жилой недвижимости', 'строительство недвижимости', 'операции с недвижимостью', 
                         'жилье', 'строительство жилой недвижимости', 'покупка коммерческой недвижимости', 
                         'операции с коммерческой недвижимостью']
true_purpose_lodging = 'жилье'
#replace_wrong_purposes(false_purpose_lodging, true_purpose_lodging)
#dataset['purpose'].value_counts()

Так как мы поменяли регистры написания одного столбца и привели цели кредита к единому образцу, то следует проверить еще раз на явные дубликаты, так как теперь они могли опять появиться:

In [14]:
print ('Дубликатов в таблице:', dataset.duplicated().sum())
dataset = dataset.drop_duplicates().reset_index(drop=True)

Дубликатов в таблице: 17


После корректировки регистров явные дубликаты появились снова, удаляем их:

In [15]:
dataset['children'].value_counts()
print ('Дубликатов в таблице:', dataset.duplicated().sum())

Дубликатов в таблице: 0


**Вывод**

Мы заменили явные дубликаты в Датасете с помощью метода drop_duplicates().reset_index(drop=True), чтобы перезаписать изначальные индексы. Так же мы исправили регистр написания данных в столбцах с помощью  .str.lower(). Неявные дубликаты мы обнаружили в столбце 'purpose', написав функцию replace_wrong_purposes мы избавились от неявных дубликатов. И в конце еще раз проверили на наличие явных дубликатов, которе появились после приведение написаня строк к нижнему регистру и правки целей кредита/

### Лемматизация

Изучая столбец 'purpose' можно заметить, что цель кредита одна и таже, а записана в разных выражения:

In [16]:
dataset['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Исправим этот момент с помощью Лемматизации и создании функции:

In [17]:
from pymystem3 import Mystem
m = Mystem()
def lemmatization(row):
    lemmas = m.lemmatize(row['purpose'])
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    if 'свадьба' in lemmas:
        return 'свадьба'
    if ('жилье' in lemmas) or ('недвижимость' in lemmas):
        return 'недвижимость'
dataset['purpose'] = dataset.apply(lemmatization, axis=1)
dataset['purpose'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose, dtype: int64

После применения Лемматизации могли появиться явные дубликаты, проверим это и удалим их если они есть:

In [18]:
print ('Дубликатов в таблице:', dataset.duplicated().sum())
dataset = dataset.drop_duplicates().reset_index(drop=True)

Дубликатов в таблице: 334


**Вывод**

С помощью Лемматизации мы привели цели кредита к единому образцу и уменьшили их количество, что позволит в дальнейшем более кофортно работать с категоризацией данных.
Так же мы проверили Датасет на явные дубликаты, которые появились после Лемматизации, и удалили их.

### Категоризация данных

Изначально мы заметили, что некоторые столбцы содержат одинаковую информацию, но записаны в разных вариантах. Например: 'education'и 'education_id' содержат информацию об уровне образования заемщиков, но в первом случае образование заполнено полностью ( текстовым выражением), во втором случае каждому уровню образования присвоена определенная категория. Посмотрим:

In [19]:
print(dataset['education'].value_counts())
dataset['education_id'].value_counts()

среднее                14876
высшее                  5212
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


1    14876
0     5212
2      744
3      282
4        6
Name: education_id, dtype: int64

Аналогочная ситуация со столбцами 'family_status' и 'family_status_id'. То есть по семейному статусу, цели кредита и образованию заемщики уже категоризированы в Датасете.
Для решения нашей задачи нам необходимо произвести категоризацию заемщиков по следующим параметрам: по уровню доходов, наличием детей.

Для начала объеденим заемнщиков в группы по количеству детей в семье. Для этого напишем функцию, которая в зависимости от количества детей, присвоит заемщику определенную группу. Создадим новый столбец: 'children_id'

* без детей
* 1-2 детей 
* многодетные

In [20]:
def children_id_def(row):
    children_group = row['children']
    if children_group == 0:
            return 'без детей'
    if children_group == 1 or children_group == 2:
            return '1-2 детей'
    if children_group > 2:
            return 'многодетные'
dataset['children_id']=dataset.apply(children_id_def, axis=1)

Вначале нашего анализа мы присвоили пропущенным значениям в колонке 'total_income' мы присвоили значение 0. Сейчас нам необходимо поменять эти значения на зарплатную медиану. Зарплатную медиану найдем в группе идентификатора уровня образования. Почему будем искать именно медиану: так как это количественные переменные, то вних есть слишком высокие и слишком малые значения, которые выбиваются из всех значений, что приведет к искажению средней величины зарплаты.

Вначале найдем зарплатную медиану в каждой группе. Необходимо учесть, что из расчета необходимо искулючить те нулевые значения, которые мы присвоили в начале пропущенным значениям.

In [21]:
higher_income=dataset[dataset['education'] == 'высшее']
higher_income=higher_income[higher_income['total_income'] !=0]
higher_income_median=higher_income['total_income'].mean().astype('int')
no_higher_income=dataset[dataset['education'] == 'неоконченное высшее']
no_higher_income=no_higher_income[no_higher_income['total_income'] !=0]
no_higher_income_median=no_higher_income['total_income'].mean().astype('int')
average_income=dataset[dataset['education'] == 'среднее']
average_income=average_income[average_income['total_income'] !=0]
average_income_median=average_income['total_income'].mean().astype('int')
school_income=dataset[dataset['education'] == 'начальное']
school_income=school_income[school_income['total_income'] !=0]
schoole_income_median=school_income['total_income'].mean().astype('int')
scientist_income=dataset[dataset['education'] == 'ученая степень']
scientist_income=scientist_income[scientist_income['total_income'] !=0]
scientist_income_median=scientist_income['total_income'].mean().astype('int')


print('Зарплатная медиана, высшее образование:', higher_income_median)
print('Зарплатная медиана, неоконченное высшее:', no_higher_income_median)
print('Зарплатная медиана, среднее образование:', average_income_median)
print('Зарплатная медиана, начальное образование:', schoole_income_median)
print('Зарплатная медиана, ученая степень:', scientist_income_median)

Зарплатная медиана, высшее образование: 207142
Зарплатная медиана, неоконченное высшее: 181533
Зарплатная медиана, среднее образование: 153715
Зарплатная медиана, начальное образование: 132155
Зарплатная медиана, ученая степень: 174749


Зная зарплатную медиану по уровню образования, мы можем заполнить нулевые значения в Датасете. Для этого поменяем вне нулевые значения в столбце на зарплатную медиану в соответствии с образованием заемщика:

In [22]:
for index in range(len(dataset)):
    if dataset['total_income'][index] == 0:
        if dataset['education'][index] == 'высшее':
            dataset['total_income'][index]=higher_income_median
for index in range(len(dataset)):
    if dataset['total_income'][index] == 0:
        if dataset['education'][index] == 'среднее':
            dataset['total_income'][index]=average_income_median          
for index in range(len(dataset)):
    if dataset['total_income'][index] == 0:
        if dataset['education'][index] == 'начальное':
            dataset['total_income'][index]=schoole_income_median            
for index in range(len(dataset)):
    if dataset['total_income'][index] == 0:
        if dataset['education'][index] == 'ученая степень':
            dataset['total_income'][index]=scientist_income_median

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


Так же добавим столбец для группировки заемщиков по признаку задолженности - есть или нет:    

In [23]:
def debt_id_def(row):
    debt_group = row['debt']
    if debt_group == 1:
            return 'Есть задолженность'
    if debt_group == 0:
            return 'Нет задолженность'
dataset['debt_id']=dataset.apply(debt_id_def, axis=1)

Так как мы не знаем по какому региону сделана выгрузка ( Москва, Санкт-Петербург или удаленные регионы России), то четко разделить уровень доходов по сумме не получится. Я взял усредненные значения для города Москвы, будем считать что выборка основана на жителях этого города:

In [30]:
def income_id_def(row):
    income_group = row['total_income']
    if income_group < 60000:
            return 'малый уровень дохода'
    if income_group > 60000 and income_group < 120000:
            return 'средний уровень дохода'
    if income_group > 120000 and income_group < 180000:
            return 'выше среднего уровень дохода'
    if income_group > 180000:
            return 'высокий уровень дохода'    
dataset['income_id']=dataset.apply(income_id_def, axis=1)
dataset.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_id,debt_id,income_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,1-2 детей,Нет задолженность,высокий уровень дохода
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,1-2 детей,Нет задолженность,средний уровень дохода
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,без детей,Нет задолженность,выше среднего уровень дохода
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,многодетные,Нет задолженность,высокий уровень дохода
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,без детей,Нет задолженность,выше среднего уровень дохода


**Вывод**

В данном разделе мы сгруппировали заемщиков по следующим признакам:
* наличие детей
* уровень образования
* семейное положение
* уровень доходов

В дальнешем эта группировка поможет нам провести зависимость фактов возврата кредиат ( невозврата) и отнесение заемщика к кокнретному признаку из указанных группировок.

## Шаг. Ответьте на вопросы  <a id='otvety'></a>

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [31]:
pivot_table_children=pd.pivot_table(dataset,index=['children_id'], columns='debt_id', values='debt', aggfunc='count')
pivot_table_children['percent'] = pivot_table_children['Есть задолженность'] / (pivot_table_children['Есть задолженность'] 
                                  + pivot_table_children['Нет задолженность']) * 100
display(pivot_table_children)

debt_id,Есть задолженность,Нет задолженность,percent
children_id,,,
1-2 детей,647,6266,9.359178
без детей,1061,12768,7.672283
многодетные,31,347,8.201058


**Вывод**


Анализируя полученную сводную таблицу, можно сказать что наибольшая доля заемщиков, которые имеют задолженность по кредиту - эти люди, которые имеют 1-2 детей.
Как ни странно люди не имеющие детей - имеют наименьший процент задолженности по кредиту.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [26]:
pivot_table_famaly=pd.pivot_table(dataset,index=['family_status'], columns='debt_id', values='debt', aggfunc='count')
pivot_table_famaly['percent'] = pivot_table_famaly['Есть задолженность'] / (pivot_table_famaly['Есть задолженность'] 
                                + pivot_table_famaly['Нет задолженность']) * 100

display(pivot_table_famaly)

debt_id,Есть задолженность,Нет задолженность,percent
family_status,,,
Не женат / не замужем,274,2510,9.841954
в разводе,85,1108,7.124895
вдовец / вдова,63,880,6.680806
гражданский брак,388,3736,9.408341
женат / замужем,929,11147,7.692945


**Вывод**

Люди, живущие в гражданском браке и не состоящие в браке, имеют наибольший процент задолженности по отношению к остальным группам людей.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [27]:
pivot_table_income=pd.pivot_table(dataset,index=['income_id'], columns='debt_id', values='debt', aggfunc='count')
pivot_table_income['percent'] = pivot_table_income['Есть задолженность'] / (pivot_table_income['Есть задолженность'] 
                                + pivot_table_income['Нет задолженность']) * 100

display(pivot_table_income)

debt_id,Есть задолженность,Нет задолженность,percent
income_id,,,
высокий уровень дохода,504,6414,7.285343
выше среднего уровень дохода,681,6607,9.344127
малый уровень дохода,52,823,5.942857
средний уровень дохода,502,5537,8.312635


**Вывод**

Как странно бы не звучало, люди, имещие уровень дохода выше среднего, имеющие наибольший процент задолженности.
Люди с малым уровнем доходом более правильно распоряжаются своими финансами - у них самый маленький процент задолженности по кредитам.

- Как разные цели кредита влияют на его возврат в срок?

In [28]:
pivot_table_purpose=pd.pivot_table(dataset,index=['purpose'], columns='debt_id', values='debt', aggfunc='count')
pivot_table_purpose['percent'] = pivot_table_purpose['Есть задолженность'] / (pivot_table_purpose['Есть задолженность'] 
                                 + pivot_table_purpose['Нет задолженность']) * 100

display(pivot_table_purpose)

debt_id,Есть задолженность,Нет задолженность,percent
purpose,,,
автомобиль,402,3870,9.410112
недвижимость,781,9797,7.383248
образование,370,3594,9.334006
свадьба,186,2120,8.065915


**Вывод**

Люди, берущие кредит на автомобиль и на образование, - самые злостные неплательщики кредита из изученной нами выборки. 
Люди, люди покупающие недвижимость, - самые ответственные.

## Шаг. Общий вывод <a id='vyvod'></a>

Проведя анализ данных, которы нам предоставил кредитный отдел банка, мы можем выделить следующие ключевые моменты:

* Люди, имеющие 1-2 ребенка отдают кредит неохотнее, за ними числется больше всего задолженности, среди других заемщиков
* Неженатые и живущие в гражданском браке заемщики - не самые лучшие кандидаты к выдаче кредита. Их процен задолжености самый высокий, в отличие от людей, состоящих в браке или которе в разводе.
* Кандидаты на кредит со средним доходом и выше среднего - чаще всего являются должниками по возврату кредита.
* Образование и автомобиль - не лучшие цели для кредита, по ним больше всего задолженности.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.